In [9]:
import sksurv as sks
import sksurv.preprocessing
import sksurv.metrics
import sksurv.datasets
import sksurv.linear_model
import sksurv.ensemble

from pathlib import Path
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import sklearn as skl
import scipy.stats as sp

import pymc as pm
import pymc_bart as pmb
import pandas as pd

import importlib
import mlflow as ml
import simsurv_func as ssf
import subprocess

In [18]:
lung = pd.read_csv("lung.csv")
lung["sex"] = lung["sex"]-1

In [19]:
y_sk = ssf.get_y_sklearn(lung["status"], lung["time"])
x_sk = lung[["sex","age","ph.karno"]]

# tranform data long-form
b_tr_t, b_tr_delta, b_tr_x = ssf.surv_pre_train2(x_sk, y_sk)
# b_te_t, b_te_x = surv_pre_test(x_sk, y_sk)
b_te_mat = pd.concat(
# b_te_x = ssf.get_bart_test(x_sk, np.unique(b_tr_t))

off = sp.norm.ppf(np.mean(b_tr_delta))


In [39]:
xx = np.concatenate([x_sk, x_sk],0)
xx[:,0] = np.concatenate([np.zeros(int(xx.shape[0]/2)), np.ones(int(xx.shape[0]/2))])
# xx[:,0] = 
# b_te_x = xx
b_te_x = ssf.get_bart_test(xx, np.unique(b_tr_t))
b_te_x.shape


(84816, 4)

In [51]:
b_tr_x = np.nan_to_num(b_tr_x, nan=90)

In [52]:
with pm.Model() as bart:
    x_data = pm.MutableData("x", b_tr_x)
    f = pmb.BART("f", X=x_data, Y=b_tr_delta, m=400, alpha = 0.99, beta=3,
            split_rules=eval("[pmb.ContinuousSplitRule(), pmb.OneHotSplitRule(), pmb.OneHotSplitRule(),pmb.OneHotSplitRule(), pmb.OneHotSplitRule(), pmb.OneHotSplitRule(), pmb.OneHotSplitRule()]"))
    z = pm.Deterministic("z", f + off)
    mu = pm.Deterministic("mu", pm.math.invprobit(z))
    y_pred = pm.Bernoulli("y_pred", p=mu, observed=b_tr_delta, shape=x_data.shape[0])
    bdata = pm.sample(random_seed=2, draws=200, tune = 200, cores=4)



Only 200 samples in chain.
Multiprocess sampling (4 chains in 4 jobs)
PGBART: [f]


Sampling 4 chains for 200 tune and 124 draw iterations (800 + 496 draws total) took 895 seconds.
/home/twj8/.local/lib/python3.10/site-packages/arviz/utils.py:184: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  numba_fn = numba.jit(**self.kwargs)(self.function)


In [ ]:
with bart:
# pm.set_data({"x":pd.DataFrame(test_x), "off":off_test})
    pm.set_data({"x":pd.DataFrame(b_te_x)})
    pp = pm.sample_posterior_predictive(bdata, var_names = ["y_pred", "f", "z", "mu"])

# transform to survival
bart_sv_fx = ssf.get_sv_fx(pp, x_out)
# bart_svt
bart_sv_t = np.unique(b_tr_t)

# add a time 0 with prob 1 
bart_sv_t = np.concatenate([np.array([0]), bart_sv_t])
bart_sv_val = [np.concatenate([np.array([1]), sv]) for sv in bart_sv_fx]

# log artif plot curves
title = "bart_surv_pred"
ssf.plot_sv(x_mat, bart_sv_val, t=bart_sv_t, title=title, save=False, show=True, dir="outputs")
